In [89]:
#export
import json
import sys
import os
import os.path
from pathlib import Path
from subprocess import call
import subprocess
import sys # system libraries, like arguments (argv)
import re # regular expressions
import pandas as pd
import glob
import os.path
from pathlib import Path



from timeit import default_timer as timer
import time



In [90]:
#export
def makeFolders(folder_list):
    for directory in folder_list:
        if not directory: # Make sure not an empty string ""
            continue
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)

In [91]:
s = "NODE_8_length_751_"
s = s.split("length")[1]
s = s.split("_")[1]
s

'751'

In [84]:
#export
class Pileup(): 
    def __init__(self, NF_out,  explicit=False, map_quality=15):
        self.explicit = explicit
        self.NF_out = NF_out
        self.wkdir = f'{self.NF_out}/unmapped/final'
        self.df_norm = pd.DataFrame()
#         self.col_data = col_data
#         self.df_col = pd.read_csv(self.col_data,sep="\t",names = ["sample","condition"])
#         self.condition_list = self.df_col["condition"].unique()
#         self.levels = ["all","single","group"]
        self.levels = ["all"]

    def pe(self, s):
        if self.explicit:
            print(s) 
            
   
            
    def get_star_output(self):
        """This function will return a df from star of all reads"""
        starpath = f'{self.NF_out}/qc/star_mapstats'
        filelist = glob.glob(starpath+"/*Log.final.out")
        df = pd.DataFrame(columns=["sample","input","unique","multi"])
        x = 0
        
        
        for f1 in filelist:
            with open(f1,"r") as infile:
                input_reads_list = []
                unique_reads_list = []
                multi_reads_list = []
                for line in infile:
                    if "Number of input reads" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        input_reads = line[ scl[0]:].strip("\t")
                        input_reads_list.append(int(input_reads))
                    if "Uniquely mapped reads number" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        unique_reads = line[ scl[0]:].strip("\t")
                        unique_reads_list.append(int(unique_reads))
                    if "Number of reads mapped to multiple loci" in line:
                        scl = [m.start() for m in re.finditer(r"\t",line)]  
                        multi_reads = line[ scl[0]:].strip("\t")
                        multi_reads_list.append(int(multi_reads))
                sample = os.path.basename(f1).replace("Log.final.out","")
                df.loc[x] = pd.Series({"sample":sample, "unique": unique_reads_list[0], "input":input_reads_list[0],"multi":multi_reads_list[0]})
            x = x + 1
            
        df["total_mapped"] = (df["unique"] + df["multi"]).astype(int)
        df["lowest_total_mapped"] = df["total_mapped"].min()
        df["mapped_norm"] =  df["lowest_total_mapped"] / df["total_mapped"]
 
           
        df.to_csv(starpath+"/starmapstats.txt",sep="\t",index=None)
        self.df_norm = df
                    

        
        
    def readCSV(self,f):
        names = ["contig","start","base","num_reads","read_bases","base_quality","map_quality"]
        names = ["contig","start","base","num_reads"]
        usecols=range(4)
        df = pd.read_csv(f,sep="\t",engine="python", names=names,usecols=usecols)
        return df
        
        
    def parsePileup(self):
        self.get_star_output()

        for level in self.levels:
            if self.explicit:
                self.pe(f'Working on {level} --- ')

                # Get some contigs for index
            files = glob.glob(f'{self.wkdir}/pileup/{level}/*pileup.txt')
            f = files[0]
            df = self.readCSV(f)
            df = df.drop_duplicates(subset="contig")
            df_final = pd.DataFrame()
            df_final["contig"] = df["contig"]
                
            for f in files:
                self.pe("---------------")
                filename = os.path.basename(f).replace(f"_unmapped_{level}_pileup.txt","")
                self.pe(f' Working on file {filename}')

                df = self.readCSV(f)
                df["length"] = df["contig"].str.split("length",expand=True)[1].str.split("_",expand=True)[1].astype(int)
                g = df.groupby(["contig"])
                df2 = g.sum()
                del df2["start"]
                del df2["length"]

                df_length = df[["contig","length"]]
                df_length = df_length.drop_duplicates(subset="contig")
                df = pd.merge(df_length,df2,on="contig",how="right")


                df["coverage"] = df["num_reads"] / df["length"]
                df["coverage"] = df["coverage"].astype(float)



                # Get norm from star output
                norm = self.df_norm[self.df_norm["sample"]==filename]["mapped_norm"]
                df["norm"] = float(norm)

                df["norm"] = df["norm"].astype(float)


                df["coverage_norm"] = df["coverage"] * df["norm"]

                makeFolders([f'{self.wkdir}/pileupParsedPerFile/',f'{self.wkdir}/pileupParsedPerFile/{level}'])
                out = f'{self.wkdir}/pileupParsedPerFile/{level}/{filename}_pileupParsed.txt'
                df.to_csv(out,sep="\t",index=None)
                df2 = df[["contig"]].copy()
                df2[filename] = df["coverage_norm"]

                
                
                df_final = pd.merge(df_final,df2,on="contig",how="outer")
                
            makeFolders([f'{self.wkdir}/pileupCoverageNormalized/'])
            out = f'{self.wkdir}/pileupCoverageNormalized/{level}_pileupCoverageNormalized.txt'
            df_final = df_final.fillna(0.0)
            df_final.to_csv(out,sep="\t",index=None)                

 

In [92]:

            
# NF = "/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/Z_RNAseq-Biome-master"

# # NF ="/Users/m/Google_Drive/Scripts/2019/biome/biome_shared/general/datasets/trem2/RNAseq-Biome-master"

# NF_out = f"{NF}/NF_OUT" 
# col_data = f"{NF}/sample_table.txt"

# P = Pileup(NF_out=NF_out,  
#              explicit=True,
#             )

# P.get_star_output()
# P.parsePileup()    

Working on all --- 
---------------
 Working on file LP00023_GT17-02957_ATTCAGAA-GTCAGTAC_S17_L005
---------------
 Working on file LP00086_GT17-02929_CGCTCATT-GCCTCTAT_S12_L003
---------------
 Working on file LP00102_GT17-02921_ATTACTCG-TAAGATTA_S7_L002
---------------
 Working on file LP00017_GT17-02941_CGCTCATT-GTCAGTAC_S22_L006
---------------
 Working on file LP00260_GT17-03011_TCCGGAGA-GCCTCTAT_S24_L006
---------------
 Working on file LP00069_GT17-02951_GAATTCGT-TCAGAGCC_S26_L007
---------------
 Working on file LP00270_GT17-02995_AGCGATAG-GCCTCTAT_S7_L002
---------------
 Working on file LP00043_GT17-02964_CTGAAGCT-AGGATAGG_S24_L006
---------------
 Working on file LP0R039_GT17-03001_TCTCGCGC-TAAGATTA_S15_L004
---------------
 Working on file LP00221_GT17-02971_TAATGCGC-TAAGATTA_S25_L007
---------------
 Working on file LP0R156_GT17-03021_ATTACTCG-GTCAGTAC_S15_L004
---------------
 Working on file LP0R138_GT17-02989_CGGCTATG-GTCAGTAC_S1_L001
---------------
 Working on file LP

KeyboardInterrupt: 

In [86]:
#export
import argparse
def parse_arguments():
        parser = argparse.ArgumentParser(description='filter quantify and graph') 
        parser.add_argument('--Nextflow_Out', action= 'store', metavar='--Nextflow_Out') 

        args = parser.parse_args()
        return args


In [87]:
#export
if __name__=="__main__":
    args = parse_arguments()
    print("Normalizing pileup")
    NF_out = args.Nextflow_Out
    
    P = Pileup(NF_out=NF_out, 
             explicit=True,
            )
    P.parsePileup()

usage: ipykernel_launcher.py [-h] [--Nextflow_Out --Nextflow_Out]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/m/Library/Jupyter/runtime/kernel-3001c855-e971-41c5-b457-a882063e7795.json


SystemExit: 2

In [88]:
import fire
!python3 notebook2script.py 2.0-PileupNormalize.ipynb

Converted 2.0-PileupNormalize.ipynb to nb_2.0-PileupNormalize.py
